In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# Cargar datos
data_path = '/Users/luissalamanca/Desktop/Duoc/Machine/ML_Proyecto_Semestral/data/03_features/engineered_data.csv'
data = pd.read_csv(data_path, sep=';')

# Separar columnas concatenadas
if len(data.columns) == 1:
    column_name = data.columns[0]
    if ',' in column_name:
        new_columns = column_name.split(',')
        data_split = data[column_name].str.split(',', expand=True)
        data_split.columns = new_columns
        for col in data_split.columns:
            data_split[col] = pd.to_numeric(data_split[col], errors='coerce')
        data = data_split

# Lista de features y target
features = [
    'EconomicEfficiency',
    'EffectivenessScore',
    'EquipmentAdvantage',
    'KillAssistRatio',
    'StealthKillsRatio'
]
X = data[features]
y = data['KDA']

# Dividir en entrenamiento/prueba (30% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [2]:
pipeline_mlp = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', MLPRegressor(
        max_iter=500,
        random_state=42
    ))
])

param_mlp = {
    'regressor__hidden_layer_sizes': [(50,), (100,), (100, 50)],
    'regressor__alpha': [0.0001, 0.001],
    'regressor__learning_rate_init': [0.001, 0.01]
}

gs_mlp = GridSearchCV(
    estimator=pipeline_mlp,
    param_grid=param_mlp,
    cv=5,
    n_jobs=-1,
    scoring='r2',
    verbose=1
)

# Ajustar GridSearchCV para MLP
gs_mlp.fit(X_train, y_train)

# Mejor modelo y predicción
best_mlp = gs_mlp.best_estimator_
y_pred_mlp = best_mlp.predict(X_test)

# Métricas
r2_mlp = r2_score(y_test, y_pred_mlp)
mse_mlp = mean_squared_error(y_test, y_pred_mlp)
rmse_mlp = np.sqrt(mse_mlp)
mae_mlp = mean_absolute_error(y_test, y_pred_mlp)

print("\n=== MLPREGRESSOR (Red Neuronal) ===")
print("Mejores hiperparámetros encontrados:")
print(gs_mlp.best_params_)
print(f"R² (Coef. determinación): {r2_mlp:.4f}")
print(f"MSE:  {mse_mlp:.4f}")
print(f"RMSE: {rmse_mlp:.4f}")
print(f"MAE:  {mae_mlp:.4f}")

Fitting 5 folds for each of 12 candidates, totalling 60 fits

=== MLPREGRESSOR (Red Neuronal) ===
Mejores hiperparámetros encontrados:
{'regressor__alpha': 0.0001, 'regressor__hidden_layer_sizes': (50,), 'regressor__learning_rate_init': 0.001}
R² (Coef. determinación): 0.8837
MSE:  0.0847
RMSE: 0.2911
MAE:  0.1768
